In [1]:
#!/usr/bin/env python
# coding: utf-8

"""
In this notebook, we:
1. Connect to MongoDB and read the four cleaned datasets.
2. Perform various analysis steps:
   - Summaries and data checks
   - Aggregations and group-bys for insights
   - Merging or joining data if beneficial for certain analyses
   - Computing correlations or other statistical measures

After this analysis, we will have a better understanding of the data and can proceed
to create visualizations in the next step.
"""

import pandas as pd
import numpy as np
from pymongo import MongoClient
from dotenv import load_dotenv
import os

# ----------------------------------------------------
# Load environment variables for MongoDB URI
# ----------------------------------------------------
load_dotenv()
MONGO_URI = os.getenv("MONGO_URI")
if not MONGO_URI:
    raise ValueError("MONGO_URI not set in .env file.")

# ----------------------------------------------------
# Connect to MongoDB and Load Data
# ----------------------------------------------------
client = MongoClient(MONGO_URI)
db = client["my_database"]

# Collections created previously:
collections = {
    "covid_vacc_death_rate": "covid_vacc_death_rate",
    "covid_vacc_manufacturer": "covid_vacc_manufacturer",
    "oecd_health_expenditure": "oecd_health_expenditure",
    "us_death_rates": "us_death_rates"
}

# Convert each collection to a DataFrame
def mongo_to_df(collection_name):
    data = list(db[collection_name].find({}, {"_id":0}))  # exclude _id for cleanliness
    return pd.DataFrame(data)

df_vdr = mongo_to_df(collections["covid_vacc_death_rate"])     # Vaccinations vs Death Rate
df_vm = mongo_to_df(collections["covid_vacc_manufacturer"])    # Vaccine Manufacturer Data
df_oecd = mongo_to_df(collections["oecd_health_expenditure"])  # OECD Health Expenditure
df_us = mongo_to_df(collections["us_death_rates"])             # US Death Rates

# ----------------------------------------------------
# Initial Exploration
# ----------------------------------------------------
print("### COVID Vaccinations vs Death Rate ###")
print(df_vdr.head())
print(df_vdr.info())
print(df_vdr.describe(include='all'))

print("\n### COVID Vaccine Manufacturer ###")
print(df_vm.head())
print(df_vm.info())
print(df_vm.describe(include='all'))

print("\n### OECD Health Expenditure ###")
print(df_oecd.head())
print(df_oecd.info())
print(df_oecd.describe(include='all'))

print("\n### US Death Rates ###")
print(df_us.head())
print(df_us.info())
print(df_us.describe(include='all'))

# ----------------------------------------------------
# Data Cleaning/Validation Checks (If Needed)
# ----------------------------------------------------
# For analysis, we might need certain columns converted to datetime or numeric if not done.
# Check if 'day' columns are datetime, if not, convert them:
for df in [df_vdr, df_vm, df_us]:
    if 'day' in df.columns:
        df['day'] = pd.to_datetime(df['day'], errors='coerce')

# Ensure year_from_day or time_period are numeric if applicable:
if 'time_period' in df_oecd.columns:
    df_oecd['time_period'] = pd.to_numeric(df_oecd['time_period'], errors='coerce')

# Check for null values and consider dropping or filling:
print("\nNull value counts in df_vdr:\n", df_vdr.isnull().sum())
print("\nNull value counts in df_vm:\n", df_vm.isnull().sum())
print("\nNull value counts in df_oecd:\n", df_oecd.isnull().sum())
print("\nNull value counts in df_us:\n", df_us.isnull().sum())

# Depending on your analysis goals, you may decide to drop rows with too many nulls
# or fill with zero/mean. We'll leave them as is for now, but note this step for refinement.

# ----------------------------------------------------
# Example Analyses
# ----------------------------------------------------

# 1. COVID Vaccinations vs Death Rate (df_vdr)
# Let's see which entities (countries) have the highest cumulative vaccination rate.
if 'covid_19_doses_cumulative,_per_hundred' in df_vdr.columns and 'entity' in df_vdr.columns:
    avg_vacc = (df_vdr.groupby('entity')['covid_19_doses_cumulative,_per_hundred']
                .mean()
                .sort_values(ascending=False))
    print("\nTop 10 entities by average cumulative vaccination (per hundred):")
    print(avg_vacc.head(10))

# Check correlation between vaccination and death rate if both columns exist:
if 'covid_19_doses_cumulative,_per_hundred' in df_vdr.columns and 'daily_new_confirmed_deaths_due_to_covid_19_per_million_people_rolling_7_day_average,_right_aligned' in df_vdr.columns:
    corr_value = df_vdr[['covid_19_doses_cumulative,_per_hundred',
                         'daily_new_confirmed_deaths_due_to_covid_19_per_million_people_rolling_7_day_average,_right_aligned']].corr().iloc[0,1]
    print(f"\nCorrelation between vaccine doses and daily death rate: {corr_value}")

# 2. COVID Vaccine Manufacturer (df_vm)
# Identify which vaccine manufacturer has the highest cumulative doses globally:
manufacturer_cols = [c for c in df_vm.columns if 'manufacturer' in c]
if manufacturer_cols:
    global_sums = df_vm[manufacturer_cols].sum().sort_values(ascending=False)
    print("\nTotal cumulative doses by manufacturer (across all entities):")
    print(global_sums)

# 3. OECD Health Expenditure (df_oecd)
# Let's check average health expenditure (obs_value) by country over time.
if 'reference_area' in df_oecd.columns and 'obs_value' in df_oecd.columns:
    avg_expenditure = df_oecd.groupby('reference_area')['obs_value'].mean().sort_values(ascending=False)
    print("\nTop 10 reference areas by average health expenditure (obs_value):")
    print(avg_expenditure.head(10))

# 4. US Death Rates (df_us)
# Check a particular age group's trends over time. Let's say "80+" if it exists.
if 'entity' in df_us.columns and df_us['entity'].eq('80+').any():
    # Filter data for 80+
    df_80plus = df_us[df_us['entity'] == '80+'].copy()
    df_80plus = df_80plus.sort_values('day')
    if 'death_rate_weekly_of_unvaccinated_people__united_states,_by_age' in df_80plus.columns:
        print("\nFirst 5 rows for 80+ age group death rates over time:")
        print(df_80plus[['day', 'death_rate_weekly_of_unvaccinated_people__united_states,_by_age']].head())

# ----------------------------------------------------
# Potential Data Integration for Analysis
# ----------------------------------------------------
# If needed, you can attempt to integrate datasets. For example, if you have a common country code:
# This depends heavily on whether keys match. If not, skip this step.

# Example: If 'entity' in df_vdr and 'reference_area' in df_oecd correspond to the same countries (just an example),
# you could try merging on a year-from-day and reference_area/time_period basis.
# Note: This might result in NaNs if keys don't align well.

# Only do this if it makes sense for your analysis:
# if 'day' in df_vdr.columns:
#     df_vdr['year'] = df_vdr['day'].dt.year
# if 'time_period' in df_oecd.columns:
#     # Attempt a merge (just as an example)
#     merged_analysis = pd.merge(df_vdr, df_oecd,
#                                left_on=['entity','year'],
#                                right_on=['reference_area','time_period'],
#                                how='inner')
#     print("\nMerged dataset shape:", merged_analysis.shape)
#     # From here, you could check correlation between health expenditure and vaccination/death rates.


# ----------------------------------------------------
# Correlations and Statistical Measures
# ----------------------------------------------------
# Depending on your numeric columns, try a correlation matrix:
print("\nCorrelation matrix for df_vdr numeric columns:")
print(df_vdr.select_dtypes(include=[np.number]).corr())

# Similar for df_vm, df_oecd, df_us:
print("\nCorrelation matrix for df_vm numeric columns:")
print(df_vm.select_dtypes(include=[np.number]).corr())

print("\nCorrelation matrix for df_oecd numeric columns:")
print(df_oecd.select_dtypes(include=[np.number]).corr())

print("\nCorrelation matrix for df_us numeric columns:")
print(df_us.select_dtypes(include=[np.number]).corr())

# ----------------------------------------------------
# Summary and Next Steps
# ----------------------------------------------------
# By now we have:
# - Basic stats and shapes
# - Grouped averages and correlations
# - Checked if merging is feasible
#
# Next steps:
# 1. Identify interesting comparisons or patterns to visualize.
# 2. Prepare subsets of data for plotting in Visualization.ipynb.

print("\nAnalysis complete. You can now proceed to Visualization steps.")


### COVID Vaccinations vs Death Rate ###
        entity code    year         day  \
0  Afghanistan  AFG  2020.0  2020-01-09   
1     Paraguay  PRY  2020.0  2020-01-09   
2         Peru  PER  2020.0  2020-01-09   
3  Philippines  PHL  2020.0  2020-01-09   
4    Australia  AUS  2020.0  2020-01-09   

   daily_new_confirmed_deaths_due_to_covid_19_per_million_people_rolling_7_day_average,_right_aligned  \
0                                                0.0                                                    
1                                                0.0                                                    
2                                                0.0                                                    
3                                                0.0                                                    
4                                                0.0                                                    

   covid_19_doses_cumulative,_per_hundred world_regions_according_to_